In [1]:
DATA_NAME = 'coco-outdoor' 
TRANSFORM = 'learned'
CHANNEL = ''

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_learned_outdoor_coco.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,366.11,0.00,-0.01,-0.01,0.01,0.00,0.00,0.01,0.0,-0.01,...,-0.0,0.0,0.0,-0.0,0.0,-0.00,-0.00,-0.0,0.00,-0.00
1,0.00,411.62,-0.01,-0.01,-0.00,0.00,0.01,0.01,-0.0,0.00,...,-0.0,-0.0,-0.0,-0.0,-0.0,0.00,-0.00,0.0,0.00,-0.00
2,-0.01,-0.01,443.94,0.01,0.02,-0.01,0.01,-0.01,-0.0,-0.01,...,-0.0,-0.0,-0.0,0.0,0.0,-0.00,0.00,-0.0,0.00,0.00
3,-0.01,-0.01,0.01,295.03,-0.01,-0.02,-0.00,0.00,0.0,-0.02,...,-0.0,0.0,0.0,0.0,-0.0,0.00,0.00,-0.0,-0.00,-0.00
4,0.01,-0.00,0.02,-0.01,525.61,-0.01,-0.01,0.00,0.0,0.00,...,-0.0,0.0,0.0,0.0,0.0,-0.00,-0.00,0.0,0.00,-0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,-0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,-0.00,0.0,0.00,...,0.0,-0.0,0.0,0.0,-0.0,36.66,0.00,-0.0,-0.00,-0.00
60,-0.00,-0.00,0.00,0.00,-0.00,-0.00,-0.00,-0.00,0.0,0.00,...,-0.0,-0.0,0.0,0.0,-0.0,0.00,3.63,-0.0,0.00,0.00
61,-0.00,0.00,-0.00,-0.00,0.00,-0.00,0.00,-0.00,-0.0,-0.00,...,-0.0,-0.0,-0.0,-0.0,0.0,-0.00,-0.00,8.5,0.00,0.00
62,0.00,0.00,0.00,-0.00,0.00,-0.00,-0.00,0.00,0.0,-0.00,...,-0.0,-0.0,-0.0,0.0,-0.0,-0.00,0.00,0.0,11.63,-0.00


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,1.00000,0.00001,-0.00001,-0.00002,0.00003,0.00000,0.00001,0.00003,0.00001,-0.00003,...,-0.00000,0.00001,0.00000,-0.00002,0.00000,-0.00000,-0.00000,-0.00000,0.00000,-0.00000
1,0.00001,1.00000,-0.00002,-0.00002,-0.00000,0.00000,0.00004,0.00005,-0.00000,0.00002,...,-0.00001,-0.00002,-0.00000,-0.00002,-0.00001,0.00001,-0.00000,0.00001,0.00001,-0.00000
2,-0.00001,-0.00002,1.00000,0.00004,0.00004,-0.00003,0.00005,-0.00005,-0.00002,-0.00003,...,-0.00000,-0.00000,-0.00001,0.00001,0.00000,-0.00001,0.00000,-0.00001,0.00001,0.00000
3,-0.00002,-0.00002,0.00004,1.00000,-0.00003,-0.00005,-0.00000,0.00002,0.00003,-0.00007,...,-0.00001,0.00000,0.00001,0.00001,-0.00001,0.00000,0.00000,-0.00001,-0.00000,-0.00000
4,0.00003,-0.00000,0.00004,-0.00003,1.00000,-0.00004,-0.00002,0.00000,0.00003,0.00000,...,-0.00001,0.00001,0.00001,0.00000,0.00001,-0.00001,-0.00001,0.00000,0.00001,-0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,-0.00000,0.00001,-0.00001,0.00000,-0.00001,0.00000,0.00001,-0.00000,0.00000,0.00001,...,0.00003,-0.00002,0.00001,0.00001,-0.00006,1.00000,0.00001,-0.00000,-0.00001,-0.00000
60,-0.00000,-0.00000,0.00000,0.00000,-0.00001,-0.00001,-0.00000,-0.00001,0.00001,0.00000,...,-0.00003,-0.00000,0.00001,0.00003,-0.00003,0.00001,1.00000,-0.00001,0.00003,0.00001
61,-0.00000,0.00001,-0.00001,-0.00001,0.00000,-0.00001,0.00000,-0.00001,-0.00000,-0.00001,...,-0.00001,-0.00002,-0.00000,-0.00000,0.00000,-0.00000,-0.00001,1.00000,0.00002,0.00001
62,0.00000,0.00001,0.00001,-0.00000,0.00001,-0.00001,-0.00001,0.00001,0.00001,-0.00000,...,-0.00002,-0.00002,-0.00000,0.00000,-0.00000,-0.00001,0.00003,0.00002,1.00000,-0.00001


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.26754351581935404

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[8.30540468e+03 4.32812303e+03 3.09408739e+03 2.68276776e+03
 2.63555446e+03 2.53437712e+03 2.42505957e+03 2.08882706e+03
 1.70760596e+03 1.36563319e+03 1.28076460e+03 1.15153514e+03
 1.04490612e+03 7.44549650e+02 6.68091659e+02 6.41520443e+02
 6.15801401e+02 5.97735118e+02 5.52594831e+02 4.94855564e+02
 4.82029422e+02 4.50499155e+02 4.32405610e+02 4.30020391e+02
 4.03571981e+02 3.33062721e+02 3.12900141e+02 3.09864826e+02
 3.04955544e+02 2.98143936e+02 2.91111051e+02 2.75763246e+02
 2.59366279e+02 2.52060221e+02 2.38015047e+02 2.24784813e+02
 1.97771122e+02 1.54933911e+02 1.46852568e+02 1.43087348e+02
 1.28929408e+02 1.06917945e+02 1.00964496e+02 9.84041487e+01
 8.63006644e+01 7.23217018e+01 6.34936385e+01 5.78435032e+01
 4.89345210e+01 3.79435594e+01 3.51898103e+01 3.34485949e+01
 3.15860822e+01 2.44135153e+01 1.73851824e+01 1.06423192e+01
 8.39351620e+00 6.55068345e+00 4.83172194e+00 3.16483839e+00
 1.63550123e+00 6.65867239e-01 1.57134692e-01 9

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,-0.009721,-0.012129,-0.014026,-0.007016,-0.021989,-0.008299,-0.003438,-0.004317,-0.000470,-0.003454,...,-0.000897,-0.000538,-0.000978,-0.000437,-0.000351,-0.000729,-0.000073,-0.000167,-0.000231,-0.000052
1,-0.017945,-0.027209,-0.039840,-0.010897,0.996977,-0.013884,-0.004607,-0.005942,-0.000589,-0.004583,...,-0.001140,-0.000675,-0.001248,-0.000555,-0.000442,-0.000924,-0.000091,-0.000212,-0.000290,-0.000066
2,-0.028965,-0.077767,0.989553,-0.013035,0.030803,-0.018635,-0.004688,-0.006360,-0.000591,-0.004754,...,-0.001123,-0.000667,-0.001238,-0.000545,-0.000436,-0.000912,-0.000090,-0.000210,-0.000285,-0.000065
3,-0.008619,0.799608,0.013065,-0.003029,0.004035,-0.004430,-0.000931,-0.001244,-0.000118,-0.000956,...,-0.000221,-0.000140,-0.000245,-0.000113,-0.000086,-0.000178,-0.000018,-0.000040,-0.000056,-0.000013
4,-0.049402,0.524471,0.062184,-0.016144,0.020593,-0.025382,-0.005102,-0.006923,-0.000617,-0.005115,...,-0.001195,-0.000704,-0.001312,-0.000579,-0.000465,-0.000957,-0.000095,-0.000221,-0.000301,-0.000070
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,-0.008105,-0.007213,-0.006679,-0.010067,-0.005647,-0.008976,-0.018074,-0.014736,-0.194037,-0.018078,...,-0.072684,-0.151817,-0.065136,-0.225346,-0.459070,-0.094932,0.057863,0.197950,0.536488,0.040775
60,0.005592,0.004975,0.004608,0.006942,0.003896,0.006194,0.012439,0.010148,0.105780,0.012438,...,0.047706,0.088584,0.043133,0.116920,0.172015,0.060458,-0.082734,-0.563372,0.740707,-0.056401
61,0.007365,0.006552,0.006071,0.009144,0.005132,0.008157,0.016352,0.013354,0.122992,0.016356,...,0.060937,0.106286,0.055392,0.133088,0.175693,0.075919,-0.340710,0.757529,0.336266,-0.204191
62,0.001888,0.001679,0.001556,0.002342,0.001316,0.002091,0.004188,0.003421,0.029692,0.004187,...,0.015383,0.026022,0.014011,0.031830,0.040302,0.018989,0.760823,0.094275,0.064122,-0.631829


In [13]:
cos_dist = spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine')
cos_dist = np.sort(np.concatenate(cos_dist))[:len(GROUPS)]
cos_dist


array([0.00093728, 0.0030231 , 0.01044744, 0.01363961, 0.01753265,
       0.02421769, 0.03056408, 0.03549079, 0.03775449, 0.04681292,
       0.04838893, 0.05070645, 0.0706726 , 0.08323147, 0.09279065,
       0.09741335, 0.11761866, 0.11954224, 0.13130811, 0.16543439,
       0.18302472, 0.18535344, 0.1895267 , 0.20039208, 0.20114972,
       0.20297462, 0.20330332, 0.21555152, 0.22726103, 0.23135961,
       0.23513051, 0.23528879, 0.23917721, 0.24104387, 0.24247077,
       0.24348031, 0.24379437, 0.24571365, 0.24768821, 0.25532235,
       0.25587019, 0.25929301, 0.26092724, 0.26463753, 0.26601522,
       0.2789985 , 0.27926319, 0.28957333, 0.31277114, 0.31546064,
       0.32669412, 0.32831246, 0.32867994, 0.33872435, 0.3402642 ,
       0.34588035, 0.38039946, 0.41571039, 0.42738682, 0.44164576,
       0.45791097, 0.45937146, 0.46351197, 0.47133689])